In [2]:
import warnings
warnings.filterwarnings("ignore")
import pathlib
import os
from src.midi_processing import get_song_vector, DOC2VEC_MODELS_PATHS
import numpy as np
from sklearn.manifold import TSNE
import joblib
from tqdm import tqdm
import multiprocessing
import matplotlib.pyplot as plt

models = {name: joblib.load(os.path.join(DOC2VEC_MODELS_PATHS, f'{name}_model.jblib')) for name in ['drums', 'melody', 'harmony']}

In [3]:
midi_files = pathlib.Path('Data/midi_files/')
all_midi_paths = list(midi_files.glob('*'))

In [4]:
songs_vectors = []
song_names = []

for midi_file in tqdm(all_midi_paths, total=len(all_midi_paths)):
    try:
        songs_vectors.append(get_song_vector(str(midi_file), models))
        song_names.append(midi_file.name)
    except Exception as e:
        print(e)
        print(f"Invalid song: {midi_file.name}")
        continue        

  3%|██                                                                               | 16/625 [00:07<03:59,  2.54it/s]

data byte must be in range 0..127
Invalid song: Aaron_Neville_-_Tell_It_Like_It_Is.mid


 20%|███████████████▋                                                                | 123/625 [01:08<03:42,  2.26it/s]

Could not decode key with 1 flats and mode 255
Invalid song: Beastie_Boys_-_Girls.mid


 27%|█████████████████████▏                                                          | 166/625 [01:34<03:43,  2.06it/s]

...

 58%|██████████████████████████████████████████████▏                                 | 361/625 [03:14<02:17,  1.92it/s]

Could not decode key with 4 flats and mode 255
Invalid song: Dan_Fogelberg_-_Leader_of_the_Band.mid


 59%|██████████████████████████████████████████████▉                                 | 367/625 [03:17<02:01,  2.12it/s]

Could not decode key with 16 sharps and mode 1
Invalid song: David_Bowie_-_Lazarus.mid


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [05:42<00:00,  2.10it/s]


In [8]:
songs_vectors = np.array(songs_vectors)

tsne = TSNE(n_components=2, verbose=3, random_state=84)
vectors_transformed = tsne.fit_transform(songs_vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 618 samples in 0.006s...
[t-SNE] Computed neighbors for 618 samples in 0.148s...
[t-SNE] Computed conditional probabilities for sample 618 / 618
[t-SNE] Mean sigma: 3.475403
[t-SNE] Computed conditional probabilities in 0.047s
[t-SNE] Iteration 50: error = 104.7694244, gradient norm = 0.2812013 (50 iterations in 0.294s)
[t-SNE] Iteration 100: error = 104.6068726, gradient norm = 0.3478881 (50 iterations in 0.336s)
[t-SNE] Iteration 150: error = 109.5602646, gradient norm = 0.2528093 (50 iterations in 0.363s)
[t-SNE] Iteration 200: error = 108.0943680, gradient norm = 0.2930265 (50 iterations in 0.443s)
[t-SNE] Iteration 250: error = 109.7697983, gradient norm = 0.2706953 (50 iterations in 0.336s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 109.769798
[t-SNE] Iteration 300: error = 2.6244369, gradient norm = 0.0038796 (50 iterations in 0.276s)
[t-SNE] Iteration 350: error = 1.9342003, gradient norm = 0.002

In [22]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool

# output to static HTML file
output_file("Interactive-Midi-Doc2Vec.html")

data_source = ColumnDataSource(data=dict(x=vectors_transformed[:, 0], y=vectors_transformed[:, 1], song_name=song_names))
hover = HoverTool(tooltips=[
    ("song_name", "@song_name"),
])

p = figure(sizing_mode='stretch_both', title="Melody embedding TSNE visualization")
p.add_tools(hover)


# add a circle renderer with a size, color, and alpha
p.circle(source=data_source, x='x', y='y', size=10, color="navy", alpha=0.5)

# show the results
save(p)

'C:\\Msc\\DeepLearning\\Deep-Learning-Intro\\assignment3\\Interactive-Midi-Doc2Vec.html'